In [3]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [27]:
df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
df.shape

(1154112, 7)

In [28]:
old_len = len(df)

In [29]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [30]:
df.duration.mean()

19.167224093791006

In [31]:
df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

In [32]:
categorical = ['PUlocationID', 'DOlocationID']

df[categorical] = df[categorical].fillna(-1).astype('int')

In [33]:
df[categorical] = df[categorical].astype('str')

In [34]:
train_dicts = df[categorical].to_dict(orient='records')

In [35]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts) 

In [36]:
X_train.shape

(1109826, 525)

In [37]:
y_train = df.duration.values

In [38]:
len(dv.feature_names_)

525

In [39]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [40]:
y_pred = lr.predict(X_train)

In [41]:
mean_squared_error(y_train, y_pred, squared=False)

10.5285194284187

In [42]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [43]:
df_val = read_data('./data/fhv_tripdata_2021-02.parquet')

In [44]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [45]:
X_val = dv.transform(val_dicts) 

In [46]:
y_pred = lr.predict(X_val)

In [47]:
y_val = df_val.duration.values

In [48]:
mean_squared_error(y_val, y_pred, squared=False)

11.014285518755779